In [45]:
!python3 train_audio_model.py --dirs "samples/test-samples/orchestra,samples/test-samples/wolves" --model_name "test-samples" --epochs 1 --augment_folders "noise_samples,"

100%|#############################################| 4/4 [00:01<00:00,  3.61it/s]
2018-10-08 21:23:37.795002: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
gathering training data for on the fly with transforms ['samples/laughter-test/seinfeld laughter.wav', 'samples/laughter-test/snl 1 laughter.wav', 'samples/laughter-test/ultimate_1_b.wav', 'samples/laughter-test/ultimate_2_b.wav']
100%|############################################| 4/4 [06:54<00:00, 124.26s/it]
these files have already been processed ['samples/notlaughter-test/ddVE1mN5_lU.wav', 'samples/notlaughter-test/jFS9rRTXue8.wav', 'samples/notlaughter-test/seinfeld not laughter.wav', 'samples/notlaughter-test/snl 1 not laughter.wav']
shuffle the files
now we split the files
preprocess for training 1886
100%|#######################################| 1886/1886 [01:21<00:00, 17.47it/s]
preprocess for training 210
100%|###############

In [61]:
!python3 predict_audio_model.py --model_name "test-samples" --dirs "samples/test-samples/orchestra-test,samples/test-samples/wolves-test" --classes "orchestra,wolves"

100%|#############################################| 2/2 [00:00<00:00, 10.21it/s]
using model test-samples/1_1
predicting files ['samples/test-samples/orchestra-test/Aq1lkc4qfFk.wav', 'samples/test-samples/wolves-test/QnVzn-1mESU.wav']
2018-10-08 22:10:41.091601: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
translated preds ['orchestra', 'orchestra', 'orchestra', 'orchestra', 'orchestra', 'orchestra', 'orchestra', 'orchestra', 'orchestra', 'orchestra', 'orchestra', 'orchestra', 'orchestra', 'orchestra', 'orchestra', 'orchestra', 'orchestra', 'orchestra', 'orchestra', 'orchestra', 'orchestra', 'orchestra', 'orchestra', 'orchestra', 'orchestra', 'orchestra', 'orchestra', 'orchestra', 'orchestra', 'orchestra', 'orchestra', 'orchestra', 'orchestra', 'orchestra', 'orchestra', 'orchestra', 'orchestra', 'orchestra', 'orchestra', 'orchestra', 'orchestra', 'orchestra', 'orchestra', 'orchestra', '

In [4]:
from audioInput_v2 import gatherData
from audio_transforms import changeGain, addCompression, changePitch, mixWith
from pydub.effects import speedup
from pydub import AudioSegment
import random
from audioUtils import readFolderRecursive

def pickRandomItems(arr, max = 1):
    return sorted(arr, key=lambda *args: random.random())[0:random.randint(1, max)]

def makeRandomTransforms(transforms, max_number_of_transforms, audio):
    transforms_to_apply = pickRandomItems(transforms, max_number_of_transforms)
    for transform in transforms_to_apply:
        audio = transform(audio)
    
    return audio

def mixWithFolder(folder, transforms = []):
    overlays = readFolderRecursive(folder)
    max_number_of_transforms = len(transforms)
        
    def curriedMixWith(audio):
        overlay = random.choice(overlays)
        transformed_overlay = makeRandomTransforms(transforms, max_number_of_transforms, AudioSegment.from_file(overlay))
        transformed_original = makeRandomTransforms(transforms, max_number_of_transforms, audio)
        return mixWith(transformed_original, transformed_overlay)
    return curriedMixWith
    
data = gatherData(['samples/laughter/ySkafsRm9po.wav'], transforms = [
    #lambda audio: audio,
    #lambda audio: changePitch(audio, min = -0.5, max = 0.5),
    #lambda audio: changeGain(audio),
    #lambda audio: addCompression(audio),
    mixWithFolder('samples/notlaughter', [
        lambda audio: changeGain(audio, 20, 10),
        lambda audio: changePitch(audio, -0.3, max=0.3),
        lambda audio: addCompression(audio),
    ]),
])
audio = None
for d in data:
    if audio is None:
        audio = d['audio']
    else:
        audio += d['audio']
audio

transform <function <lambda> at 0x7f7f9afbbf28>
transform <function <lambda> at 0x7f7fe53fcc80>
transform <function <lambda> at 0x7f7f9afbbf28>
transform <function <lambda> at 0x7f7f9aff7048>


In [17]:
hilarious_audio_track